In [1]:
import os

In [2]:
%pwd

'/Users/apple/Downloads/Project/Wine-Quality-Prediction-End-to-End-Project/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/apple/Downloads/Project/Wine-Quality-Prediction-End-to-End-Project'

In [6]:
from dataclasses import dataclass
from pathlib import Path

# 📦 ModelTrainerConfig: মডেল ট্রেইনারের কনফিগারেশন সংরক্ষণ করার জন্য একটি ডেটাক্লাস
@dataclass(frozen=True)  # ❄️ frozen=True মানে এটি ইমিউটেবল হবে (ডেটা চেঞ্জ করা যাবে না)
class ModelTrainerConfig:
    root_dir: Path  # 🗂️ মূল ডিরেক্টরি যেখানে মডেল সংরক্ষণ হবে
    train_data_path: Path  # 🧪 ট্রেইন ডেটার পাথ
    test_data_path: Path  # 🔬 টেস্ট ডেটার পাথ
    model_name: str  # 🏷️ মডেলের নাম (যেমন: ElasticNet)
    alpha: float  # ⚙️ এলাস্টিক নেট মডেলের হাইপারপ্যারামিটার (রেগুলারাইজেশন স্ট্রেংথ)
    l1_ratio: float  # ⚖️ L1 এবং L2 রেগুলারাইজেশনের মধ্যে ভারসাম্য (0 থেকে 1 এর মধ্যে মান)
    target_column: str  # 🎯 টার্গেট কলামের নাম (যেটি প্রেডিক্ট করতে চাই)


In [7]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,       # 🛠️ মূল কনফিগ ফাইলের পাথ
        params_filepath = PARAMS_FILE_PATH,       # ⚙️ প্যারামিটার সংক্রান্ত YAML ফাইলের পাথ
        schema_filepath = SCHEMA_FILE_PATH):      # 🧬 স্কিমা ফাইলের পাথ (যেখানে টার্গেট কলাম ডেফাইন করা আছে)

        # 🗂️ YAML ফাইলগুলো থেকে কনফিগ, প্যারামস এবং স্কিমা পড়া হচ্ছে
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        # 📁 মূল আর্টিফ্যাক্ট ডিরেক্টরি তৈরি করা হচ্ছে
        create_directories([self.config.artifacts_root])


    # 🧠 এই ফাংশনটি মডেল ট্রেইনার কনফিগারেশন তৈরি করে রিটার্ন করবে
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer             # 🎯 মডেল ট্রেইনার সংক্রান্ত কনফিগ নেওয়া
        params = self.params.ElasticNet                # 🔧 এলাস্টিকনেট এর হাইপারপ্যারামিটার নেওয়া
        schema = self.schema.TARGET_COLUMN             # 🧾 টার্গেট কলামের নাম নেওয়া স্কিমা থেকে

        # 📁 মডেল ট্রেইনারের জন্য ডিরেক্টরি তৈরি
        create_directories([config.root_dir])

        # ✅ ModelTrainerConfig অবজেক্ট তৈরি
        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,                            # 📁 মূল ডিরেক্টরি
            train_data_path = config.train_data_path,            # 🧪 ট্রেইন ডেটার লোকেশন
            test_data_path = config.test_data_path,              # 🔬 টেস্ট ডেটার লোকেশন
            model_name = config.model_name,                      # 🏷️ মডেলের নাম
            alpha = params.alpha,                                # ⚙️ রেগুলারাইজেশন strength (alpha)
            l1_ratio = params.l1_ratio,                          # ⚖️ L1-L2 ratio
            target_column = schema.name                          # 🎯 টার্গেট কলামের নাম
        )

        return model_trainer_config  # 🔁 কনফিগ অবজেক্ট রিটার্ন করা হচ্ছে


In [9]:
import pandas as pd
import os
from mlProject import logger
from sklearn.linear_model import ElasticNet
import joblib

In [10]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config  # 🛠️ কনফিগারেশন অবজেক্ট সংরক্ষণ করা হচ্ছে


    def train(self):
        # 📥 ট্রেইন ও টেস্ট ডেটা CSV ফাইল থেকে লোড করা হচ্ছে
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        # 🎯 ইনপুট (X) ও টার্গেট (Y) ফিচার আলাদা করা হচ্ছে
        train_x = train_data.drop([self.config.target_column], axis=1)  # 🧪 ট্রেইন ইনপুট ফিচার
        test_x = test_data.drop([self.config.target_column], axis=1)    # 🔬 টেস্ট ইনপুট ফিচার
        train_y = train_data[[self.config.target_column]]               # 🎯 ট্রেইন টার্গেট
        test_y = test_data[[self.config.target_column]]                 # 🎯 টেস্ট টার্গেট

        # 🤖 মডেল ইনিশিয়ালাইজ করা হচ্ছে (ElasticNet রিগ্রেশন)
        lr = ElasticNet(
            alpha=self.config.alpha,            # 🔧 রেগুলারাইজেশন প্যারামিটার (alpha)
            l1_ratio=self.config.l1_ratio,      # ⚖️ L1 এবং L2 এর ভারসাম্য
            random_state=42                     # 🎲 র‍্যান্ডমনেস কনট্রোল
        )

        # 📚 মডেল ট্রেইন করা হচ্ছে
        lr.fit(train_x, train_y)

        # 💾 ট্রেইন করা মডেলটি `.pkl` ফাইলে সেভ করে রাখা হচ্ছে
        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))


In [11]:
try:
    # ⚙️ কনফিগারেশন ম্যানেজার অবজেক্ট তৈরি করা হচ্ছে
    config = ConfigurationManager()

    # 📄 মডেল ট্রেইনার কনফিগারেশন লোড করা হচ্ছে
    model_trainer_config = config.get_model_trainer_config()

    # 🤖 মডেল ট্রেইনার অবজেক্ট তৈরি করা হচ্ছে
    model_trainer = ModelTrainer(config=model_trainer_config)

    # 🎯 মডেল ট্রেইন ফাংশন কল করা হচ্ছে
    model_trainer.train()

except Exception as e:
    # ❌ কোনো এক্সসেপশন হলে সেটা রেইজ করে দেখানো হচ্ছে
    raise e


[2025-05-16 15:31:18,311: INFO: common: yaml file: config/config.yaml loaded successfully ✅]
[2025-05-16 15:31:18,318: INFO: common: yaml file: params.yaml loaded successfully ✅]
[2025-05-16 15:31:18,382: INFO: common: yaml file: schema.yaml loaded successfully ✅]
[2025-05-16 15:31:18,386: INFO: common: ✅ ডিরেক্টরি তৈরি হয়েছে: artifacts]
[2025-05-16 15:31:18,392: INFO: common: ✅ ডিরেক্টরি তৈরি হয়েছে: artifacts/model_trainer]
